In [ ]:
!pip install datasets peft transformers

In [ ]:
from datasets import load_dataset
data = load_dataset("facebook/flores","all")

In [ ]:
print(data)

In [ ]:
train_data = data['dev']

In [ ]:
test_data = data['devtest']

In [ ]:
test_data[0]

In [ ]:
from transformers import AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType
model_name_or_path = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
tokenizer_name_or_path = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

peft_config = LoraConfig(
    task_type=TaskType.CASUAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)

model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

model = AutoPeftModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T")

model.eval()
inputs = tokenizer("Preheat the oven to 350 degrees and place the cookie dough", return_tensors="pt")

outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=50)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])